In [1]:
import numpy as np
import matplotlib.pyplot as plt
import datetime
import imageio
import os

In [65]:
class Grid:
    def __init__(self, SIZE = 11, p = 0.8):
        self._p = p
        self.state = {'fire': 3, 'forest': 2, 'ash':1, 'empty':0}
        self.shape = np.array([SIZE, SIZE])
        self._grid = np.random.choice(3, SIZE*SIZE , p = [1-p, 0, p]).reshape((SIZE, SIZE))
        self._grid[SIZE//2,SIZE//2] = self.state['fire']
        self._newGrid = self._grid.copy()
        self.palette = np.array([[212,212,212],     # empty
                    [125,72,76],      # ash 
                    [0,170,0],       # forest
                   [240,8,24]])     # fire
    def _find_neighbors(self, pos):
        l = pos[0]
        c = pos[1]
        neighbors = []
        for L in range(-1,2):
            for C in range(-1,2):
                if L!=0 or C!=0:
                    if not(l+L == -1 or l+L > self.shape[0]-1 or c+C == -1 or c+C > self.shape[1]-1):
                        neighbors.append(np.array([l+L, c+C]))
        return neighbors
    
    def ignite(self, pos):
        if self._grid[pos[0],pos[1]] == self.state['forest']:
            self._newGrid[pos[0],pos[1]] = self.state['fire']
            
    def isBurning(self, pos):
        if self._grid[pos[0],pos[1]] == self.state['fire']:
            return True
        else: 
            return False
        
    def isNearFire(self, pos):
        isNearFire = False
        for neigh in self._find_neighbors(pos):
            if self._grid[neigh[0], neigh[1]] == self.state['fire']:
                isNearFire = True
        return isNearFire
        
    def turn_to_ash(self, pos):
        self._newGrid[pos[0],pos[1]] = self.state['ash']
        
    def switch_grid(self):
        self._grid = self._newGrid.copy()
        
    def isStillBurning(self):
        nb_fire = len(np.where(self._grid == self.state['fire'])[0])
        if nb_fire==0:
            return False
        else:
            return True
    def draw_grid(self):
        plt.title('p=%s'%self._p) 
        plt.imshow(palette[self._grid])
        
    def save_grid(self, epochs_bf_death):
        output_file = '%s.jpg' % datetime.datetime.now()#.strftime('%Y-%M-%d-%H-%M-%S')
        plt.title('p=%s epochs=%s'%self._p%epochs_bf_death) 
        plt.imshow(self.palette[self._grid])
        plt.savefig(output_file,bbox_inches='tight', dpi=75)

In [67]:
class World:
    def __init__(self, EPOCHS = 4):
        self.grid = Grid()
        self.EPOCHS = EPOCHS
        self.epochs_before_death = 0
        
    def one_round(self):
        sizeL = self.grid.shape[0]
        sizeC = self.grid.shape[1]
        for l in range(sizeL):
            for c in range(sizeC):
                pos = (l,c)
                if self.grid.isBurning(pos):
                    self.grid.turn_to_ash(pos)
                elif self.grid.isNearFire(pos):
                    self.grid.ignite(pos)        
        self.grid.switch_grid()
        
    def until_ash(self, save_plot = False):
        stillBruning = True
        epochs = 0
        while stillBruning and epochs < 30:
            if save_plot:
                self.save()
            self.one_round()
            stillBruning = self.grid.isStillBurning()
            epochs += 1
        self.epochs_before_death = epochs
        
    def draw(self):
        self.grid.draw_grid()
        
    def save(self, epochs_bf_death=0):
        self.grid.save_grid(epochs_bf_death)
        
    def create_gif(self, path='.', duration=0.5):
        images = []
        for filename in os.listdir(path):
            #print(filename)
            if filename[-3:] == 'jpg':
                images.append(filename)                
        if len(images) != 0:
            images.sort()
            imagesIo = []
            for im in images:
                imagesIo.append(imageio.imread(im))
                os.remove(im)
            output_file = 'Gif-%s.gif' % datetime.datetime.now().strftime('%Y-%M-%d-%H-%M-%S')
            imageio.mimsave(output_file, imagesIo, duration=duration)

In [68]:
w = World()
w.until_ash(save_plot=True)
w.create_gif()

TypeError: not enough arguments for format string

In [51]:
im = w.create_gif()

In [26]:
im.sort()
im

['2020-01-07 16:53:21.528520.jpg',
 '2020-01-07 16:53:21.637084.jpg',
 '2020-01-07 16:53:21.724913.jpg',
 '2020-01-07 16:53:21.819582.jpg',
 '2020-01-07 16:53:21.918885.jpg',
 '2020-01-07 16:53:22.025508.jpg',
 '2020-01-07 16:53:22.137734.jpg']

In [ ]:
p=0.5
SIZE=11
_grid = np.random.choice(3, SIZE*SIZE , p = [p, 0, 1-p]).reshape((SIZE, SIZE))
print(_grid)

In [ ]:
palette = np.array([[  0,   0,   0],   # black
                    [220,220,220],     # grey 
                    [  0, 255,   0],   # green
                   [255,   0,   0]])     # red
im = palette[_grid]
plt.imshow(im)